In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 24 02:48:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
import keras
from keras.layers.pooling import MaxPooling2D

from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
import pandas as pd
import csv
from tensorflow.keras.optimizers import SGD
from keras.layers import Input ,  Dense, Dropout, Flatten , Activation , ZeroPadding2D

from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.initializers import RandomNormal
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2                 
import os          
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix        
from random import shuffle
import tensorflow as tf 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob as gb
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
!pip install pydicom
from tensorflow.keras.applications import EfficientNetB7
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from functools import partial
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Conv2D,BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras import callbacks
import random
import pprint
!pip install pypng
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import layers
from keras.layers import Concatenate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 2.3 MB/s 


LOAD_DATA

In [17]:
import numpy as np
x_train = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/data_LVTN/no_cut/sum/512/X.npy')
y_train = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/data_LVTN/no_cut/sum/512/y.npy')

print(y_train.shape)
print(x_train.shape)
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.25, 
    rescale=1./4095,)
datagen.fit(x_train)

(2800, 4)
(2800, 512, 512, 1)


In [22]:
import numpy as np
x_test = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/data_LVTN/no_cut/test/128/X.npy')
y_test = np.load('/content/drive/MyDrive/ĐCLVTN/data_4classes/data_LVTN/no_cut/test/128/y.npy')
x_test = x_test/4095
print(x_test.shape)
print(y_test.shape)

(835, 128, 128, 1)
(835, 4)


**VGG_16+19**


In [ ]:
cb1 = tf.keras.callbacks.ModelCheckpoint(
    '/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/softmax/VGG19_512.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
baseModel = VGG16(input_shape=(512, 512,1), weights = None, include_top=False)
for layer in baseModel.layers:
  layer.trainable = True

headModel = baseModel.output
headModel = AveragePooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(512, activation="tanh")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(256, activation="tanh")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(4, activation='softmax')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()
opt = SGD(momentum = 0.9, learning_rate=  0.01)
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=['binary_accuracy'])

In [ ]:
import time
start = time.time()
# fits the model on batches with real-time data augmentation:
model_vgg16 = model.fit(x_train, y_train, validation_data=(x_val,y_val), callbacks = cb1, batch_size = 32,
         epochs=20)  
print("Total time: ", time.time() - start, "seconds")

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=32,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb1, 
         epochs=50)
print("Total time: ", time.time() - start, "seconds")

In [ ]:
acc = model1.history['binary_accuracy']                                                                                                         
val_acc =model1.history['val_binary_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

In [ ]:
from tensorflow import keras
loadedModel = keras.models.load_model('/content/drive/MyDrive/ĐCLVTN/image/VGG16_224_proposed.h5')
#loadedModel.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['binary_accuracy'])
loadedModel.evaluate(x_test,y_test,batch_size = 10)
y_pre = loadedModel.predict(x_test)
y_pre = np.round(y_pre)
y_pre
y_true = y_test
from sklearn.metrics import classification_report
import numpy as np

labels =[ 'A','COVID', 'NORMAL','PNEUMONIA']
print(classification_report(y_true, y_pre, target_names=labels))

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(np.asarray(y_test).argmax(axis=1), np.asarray(y_pre).argmax(axis=1))

#confusion_matrix(y_true,y_pre)
sns.heatmap(cm,cmap="Blues", annot = True,annot_kws={"size": 16},fmt='g')

RESNET_50

In [ ]:
cb_res1 = tf.keras.callbacks.ModelCheckpoint(
    '/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/softmax/RESNET50_512_1_standard.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
cb_res2 = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights = None, include_top=False, input_shape=(512,512, 1))
# Freeze the layers except the last 4 layers
for layer in base_model.layers[:]:
    layer.trainable = True
#base_model.summary()
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(base_model)
 
# Add new layers
model.add(Flatten())
model.add(Dense(512, activation='tanh'))
#model.add(Dropout(0.5))
model.add(Dense(256, activation='tanh'))
#model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

opt = SGD(momentum=0.9)
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=['binary_accuracy'])

# Show a summary of the model. Check the number of trainable parameters
model.summary()

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=16,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb_res1,
         epochs=50) 
print("Total time: ", time.time() - start, "seconds")

In [ ]:
acc = model1.history['binary_accuracy']
val_acc =model1.history['val_binary_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

In [ ]:
from tensorflow import keras
model2 = keras.models.load_model('/content/RESNET50_128_standard.h5')
model2.evaluate(x_test,y_test)
y_pre = model2.predict(x_test)
y_pre = np.round(y_pre)
y_true = y_test
from sklearn.metrics import classification_report
import numpy as np

labels =['A', 'B', 'C','D']

y_true = y_true
y_pred = y_pre

print(classification_report(y_true, y_pred, target_names=labels))
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(np.asarray(y_test).argmax(axis=1), np.asarray(y_pre).argmax(axis=1))

#confusion_matrix(y_true,y_pre)
sns.heatmap(cm,cmap="Blues", annot = True,annot_kws={"size": 16},fmt='g')

EfficientNetB1


In [ ]:
cb_eff = tf.keras.callbacks.ModelCheckpoint(
    '/content/drive/MyDrive/ĐCLVTN/data_4classes/new-4/no_cut/softmax/EFF_512.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
eff = EfficientNetB1(include_top = False, weights = None, input_shape=(512,512,1))
for layer in eff.layers:
    layer.trainable = True

In [ ]:
model = Sequential()
model.add(eff)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='tanh'))
#model.add(Dropout(0.2))
model.add(Dense(256, activation='tanh'))
#model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
opt = SGD(momentum= 0.9)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['binary_accuracy'])

model.summary()

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=16,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb_eff, 
         epochs=50) 
print("Total time: ", time.time() - start, "seconds")

In [ ]:
from tensorflow import keras
model2 = keras.models.load_model('/content/EFF_128.h5')
model2.evaluate(x_test,y_test)

In [ ]:
acc = model1.history['binary_accuracy']
val_acc =model1.history['val_binary_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs=range(len(acc))
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

In [ ]:
y_pre = model2.predict(x_test)
y_pre = np.round(y_pre)
y_pre = model2.predict(x_test)
y_pre = np.round(y_pre)
y_true = y_test
from sklearn.metrics import classification_report
import numpy as np

labels =['A','B','C','D']

y_true = y_true
y_pred = y_pre

print(classification_report(y_true, y_pred, target_names=labels))
cm = confusion_matrix(np.asarray(y_test).argmax(axis=1), np.asarray(y_pre).argmax(axis=1))
sns.heatmap(cm,cmap="Blues", annot = True,annot_kws={"size": 16},fmt='g')

INCEPTIONRESNETV2


In [18]:
cb_incep = tf.keras.callbacks.ModelCheckpoint(
    '/content/INCEP_512.h5', monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
pretrain_model = InceptionResNetV2(
        include_top=False, 
        weights=None,
        input_shape = (512,512,1))    
for layer in pretrain_model.layers:
    layer.trainable = True
model = Sequential()
model.add(pretrain_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='tanh'))
#model.add(Dropout(0.2))
model.add(Dense(256, activation='tanh'))
#model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='sgd', loss="binary_crossentropy", metrics=['binary_accuracy'])

model.summary()

In [ ]:
import time
start = time.time()
model1 = model.fit(datagen.flow(x_train, y_train, batch_size=16,subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'), callbacks = cb_incep, 
         epochs=50) 
print("Total time: ", time.time() - start, "seconds")

PREDICT

In [23]:
model2 = keras.models.load_model('/content/drive/MyDrive/ĐCLVTN/data_4classes/data_LVTN/no_cut/softmax/INCEP_128.h5')

In [ ]:
#####predict
IMG_SIZE = 224

a = cv2.imread('/content/drive/MyDrive/ĐCLVTN/0_COVID-158.png')
img_array = cv2.cvtColor(a ,cv2.COLOR_BGR2RGB)
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
a = new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

pred = model2.predict(a)
print(np.argmax(pred))

0


In [28]:
import pydicom as dicom
IMG_SIZE = 128
model = keras.models.load_model('/content/drive/MyDrive/ĐCLVTN/data_4classes/data_LVTN/no_cut/softmax/INCEP_128.h5')
dataset = dicom.read_file('/content/1_af4ab9c77eca05d706b877bb52a23303.dcm')
my_data = dataset.pixel_array
my_data2 = cv2.resize(my_data, (IMG_SIZE,IMG_SIZE))
a = my_data2.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

pred = model.predict(a)
print(pred)
print(np.argmax(pred))

[[1.3553157e-05 1.7799158e-02 9.8198801e-01 1.9920645e-04]]
2
